In [1]:
import cv2
import numpy as np
import pandas as pd
import pickle
import xgboost as xgb
#from matplotlib import pyplot as plt
import skimage
import os
import sys
from time import time
sys.path.append(os.environ['REPO_DIR'])
from extractPatches import patch_extractor
from lib.utils import configuration, run

In [2]:
def CDF(x):
    x=np.sort(x)
    size=x.shape[0]
    y=np.arange(0,size)/size
    return x,y

In [3]:
def setup_download_from_s3(rel_fp, recursive=True):
    s3_fp = 's3://mousebrainatlas-data/' + rel_fp
    local_fp = os.environ['ROOT_DIR'] + rel_fp

    if os.path.exists(local_fp):
        print('ALREADY DOWNLOADED FILE')
        return

    if recursive:
        run('aws s3 cp --recursive {0} {1}'.format(s3_fp, local_fp))
    else:
        run('aws s3 cp {0} {1}'.format(s3_fp, local_fp))
        
def setup_upload_from_s3(rel_fp, recursive=True):
    s3_fp = 's3://mousebrainatlas-data/' + rel_fp
    local_fp = os.environ['ROOT_DIR'] + rel_fp

    if recursive:
        run('aws s3 cp --recursive {0} {1}'.format(local_fp, s3_fp))
    else:
        run('aws s3 cp {0} {1}'.format(local_fp, s3_fp))

In [4]:
def features_extractor(patch,params,extractor):
    tile=patch #cv2.imread(patch,0)
    if params['preprocessing']['polarity']==-1:
        tile = 255-tile
    min_std=params['preprocessing']['min_std']
    _std = np.std(tile.flatten())

    extracted = []
    if _std < min_std:
        extracted.append([0] * 201)
        time_DM = 0
        time_transform = 0
    else:
        Stats = extractor.segment_cells(tile)
        cells = extractor.extract_blobs(Stats,tile)
        time_DM = extractor.timestamps[-1][1]-extractor.timestamps[0][1]
        time_transform = 0
        for i in range(1,len(extractor.timestamps)-1,2):
            time_transform = time_transform+extractor.timestamps[i+1][1]-extractor.timestamps[i][1]
        cells = pd.DataFrame(cells)
        cells = cells[cells['padded_patch'].notnull()]
        cells = cells.drop(['padded_patch','left','top'],1)
        cells = np.asarray(cells)
        for k in range(len(cells)):
            cells[k][0] = cells[k][0][:10]
        origin = np.concatenate((np.array(list(cells[:,0])),cells[:,1:]),axis=1)
        for k in range(origin.shape[1]):
            x, y = CDF(origin[:,k])
            ten = [x[np.argmin(np.absolute(y-0.1*(j+1)))] for j in range(10)]
            extracted.extend(ten)
        extracted.extend([cells.shape[0]/100])
    return extracted, time_DM, time_transform

In [5]:
%%time
t0=time()
stack = 'MD594'
fp = os.path.join('CSHL_data_processed', stack, stack + '_sorted_filenames.txt')
setup_download_from_s3(fp, recursive=False)
with open(os.environ['ROOT_DIR']+fp, 'r') as f:
    fn_idx_tuples = [line.strip().split() for line in f.readlines()]
    section_to_filename = {int(idx): fn for fn, idx in fn_idx_tuples}

fname = os.path.join('CSHL_data_processed', stack, 'All_patch_locations.pkl')
setup_download_from_s3(fname, recursive=False)
all_patch_locations = pickle.load(open(os.environ['ROOT_DIR']+fname, 'rb'), encoding='latin1')

fname = os.path.join('CSHL_data_processed', stack, 'Annotation.npy')
setup_download_from_s3(fname, recursive=False)
annotation = np.load(os.environ['ROOT_DIR']+fname, allow_pickle = True, encoding='latin1')
contours = pd.DataFrame(annotation)
contours = contours.rename(columns={0:"name", 1:"section", 2:"vertices"})
contours_grouped = contours.groupby('section')
print(time()-t0)

ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
0.22107887268066406
CPU times: user 161 ms, sys: 48.9 ms, total: 209 ms
Wall time: 221 ms


In [6]:
%%time
#Parameters
param = {}
param['max_depth']= 3   # depth of tree
param['eta'] = 0.2      # shrinkage parameter
param['silent'] = 1     # not silent
param['objective'] = 'binary:logistic' #'multi:softmax'
param['nthread'] = 7 # Number of threads used
param['num_class']=1
num_round = 100

yamlfile=os.environ['REPO_DIR']+'shape_params.yaml'
params=configuration(yamlfile).getParams()

cell_dir = os.environ['ROOT_DIR'] + 'CSHL_patches_features/MD589/'
raw_images_root = 'CSHL_data_processed/'+stack+'/'+stack+'_prep2_lossless_gray/'
features_fn = 'CSHL_grid_features/'
if not os.path.exists(os.environ['ROOT_DIR']+features_fn):
    os.mkdir(os.environ['ROOT_DIR']+features_fn)
features_fn = features_fn+stack+'/'
if not os.path.exists(os.environ['ROOT_DIR']+features_fn):
    os.mkdir(os.environ['ROOT_DIR']+features_fn)

savepath = 'CSHL_hsv/'
if not os.path.exists(os.environ['ROOT_DIR']+savepath):
    os.mkdir(os.environ['ROOT_DIR']+savepath)
savepath = savepath+stack+'/'
if not os.path.exists(os.environ['ROOT_DIR']+savepath):
    os.mkdir(os.environ['ROOT_DIR']+savepath)

resol = 0.46
half_size = 112

CPU times: user 4.4 ms, sys: 1.71 ms, total: 6.11 ms
Wall time: 10.8 ms


/Users/kuiqian/Github/shapeology_code/scripts/lib/utils.py:22: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.D=yaml.load(open(yamlFile,'r'))


In [7]:
params

{'name': 'local',
 'paths': {'s3stem': 's3://mousebraindata-open/MD657',
  'patches': 'permuted',
  'DiffusionMap': '/Users/kuiqian/Github/shapeology_code/notebooks/diffusionMap',
  'scripts_dir': '/Users/kuiqian/Github//shapeology_code/scripts',
  'data_dir': '/Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/'},
 'preprocessing': {'polarity': -1,
  'min_std': 10,
  'offset': -20,
  'min_area': 10},
 'normalization': {'size_thresholds': [15, 51, 201]}}

In [14]:
%%time
section = 130
img_fn = raw_images_root + section_to_filename[section] + '_prep2_lossless_gray.tif'
setup_download_from_s3(img_fn, recursive=False)
img = cv2.imread(os.environ['ROOT_DIR']+img_fn, 2)

run cmd= aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD594/MD594_prep2_lossless_gray/MD594-IHC24-2015.08.26-16.39.39_MD594_2_0071_prep2_lossless_gray.tif /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD594/MD594_prep2_lossless_gray/MD594-IHC24-2015.08.26-16.39.39_MD594_2_0071_prep2_lossless_gray.tif
CPU times: user 763 ms, sys: 642 ms, total: 1.41 s
Wall time: 25.2 s


In [15]:
%%time
m, n = img.shape


polygons = [(contour['name'], contour['vertices']) \
            for contour_id, contour in contours_grouped.get_group(section).iterrows()]

grid_fn = features_fn + str(section) + '.pkl'

CPU times: user 2.53 ms, sys: 327 µs, total: 2.86 ms
Wall time: 2.84 ms


In [16]:
%%time
extractor=patch_extractor(params)
num_cells = 0
grid_features = {}
NotUpload = True
count = 0
tb = time()
time_train = 0
time_upload = 0
time_features = 0
time_DMs = 0
time_transforms = 0
num_patch = 0
for contour_id, contour in polygons:
    t0=time()
    structure = contour_id
    if structure not in all_patch_locations[section].keys():
        continue
    polygon = contour.copy()

    if structure == '7n':
        structure = '7nn'

    subpath = savepath + structure + '/'
    if not os.path.exists(os.environ['ROOT_DIR']+subpath):
        os.mkdir(os.environ['ROOT_DIR']+subpath)
    
    t1=time()
    fp = []
    fp.append(cell_dir + structure + '/MD589_' + structure + '_positive.pkl')
    fp.append(cell_dir + structure + '/MD589_' + structure + '_negative.pkl')
    features = []
    labels = []
    for state in range(2):
        clouds = pickle.load(open(fp[state], 'rb'))
        features.extend(np.array(clouds))
        labels.extend([1 - state] * len(clouds))
    features = np.array(features)
    labels = np.array(labels)
    X_train = features
    y_train = labels
    dtrain = xgb.DMatrix(X_train, label=y_train)
    bst = xgb.train(param, dtrain, num_round, verbose_eval=False)
    time_train = time_train+time()-t1
    print('Train finished in %5.1f seconds' % (time() - t1))
    

    if structure == '7nn':
        structure = '7n'

    negative = structure + '_surround_500um_noclass'

    [left, right, up, down] = [int(max(min(all_patch_locations[section][negative][:, 0]) - half_size, 0)),
                               int(min(np.ceil(max(all_patch_locations[section][negative][:, 0]) + half_size),
                                       n - 1)),
                               int(max(min(all_patch_locations[section][negative][:, 1]) - half_size, 0)),
                               int(min(np.ceil(max(all_patch_locations[section][negative][:, 1]) + half_size),
                                       m - 1))]
    t2=time()
    xs, ys = np.meshgrid(np.arange(left + half_size, right - half_size + 1, half_size * 2),
                         np.arange(up + half_size, down - half_size + 1, half_size * 2), indexing='xy')
    locations = np.c_[xs.flat, ys.flat]
    inside = all_patch_locations[section][structure]
    all_rows = locations.view([('', locations.dtype)] * locations.shape[1])
    inside_rows = inside.view([('', inside.dtype)] * inside.shape[1])
    outside = np.setdiff1d(all_rows, inside_rows).view(locations.dtype).reshape(-1, locations.shape[1])
    windows = []
    windows.append(inside)
    windows.append(outside)
    polygon[:, 0] = polygon[:, 0] - left
    polygon[:, 1] = polygon[:, 1] - up
    print('location finished in %5.1f seconds' % (time() - t2))

    hsv = np.zeros([down - up + 1, right - left + 1, 3])
    hsv[:, :, 2] = 1
    single_time = 0
    t3=time()
    for state in range(2):
        for index in range(len(windows[state])):
            try:
                t4=time()
                x = int(float(windows[state][index][0]))
                y = int(float(windows[state][index][1]))
                patch = img[y - half_size:y + half_size, x - half_size:x + half_size].copy()
                grid_index = str(section)+'_'+str(x)+'_'+str(y)
                if grid_index in grid_features.keys():
                    extracted = grid_features[grid_index]
                else:
                    extracted, time_DM, time_transform = features_extractor(patch, params, extractor)
                    grid_features[grid_index] = extracted
                    time_DMs += time_DM
                    time_transforms += time_transform
                    num_patch += 1
                    num_cells += len(extracted)
                single_time=single_time+time()-t4
                
                xtest = xgb.DMatrix(extracted)
                score = bst.predict(xtest, output_margin=True, ntree_limit=bst.best_ntree_limit)
                value_img = patch / 255
                hsv[y - half_size - up:y + half_size - up, x - half_size - left:x + half_size - left, 2] = value_img
                satua_img = np.zeros_like(value_img) + score
                origin = hsv[y - half_size - up:y + half_size - up, x - half_size - left:x + half_size - left, 1]
                comp = np.absolute(origin) - np.absolute(satua_img)
                hsv[y - half_size - up:y + half_size - up, x - half_size - left:x + half_size - left, \
                1] = origin * (comp > 0) + satua_img * (comp < 0)
            except:
                continue
    time_features=time_features+single_time
    print('Features finished in %5.1f seconds' % (single_time))
    print('Allocation finished in %5.1f seconds' % (time() - t3))
    t5=time()
    hsv[:, :, 0] = (hsv[:, :, 1] < 0) * 0.66 + (hsv[:, :, 1] > 0) * 1.0
    hsv[:, :, 1] = np.absolute(hsv[:, :, 1])
    hsv[:, :, 1] = (hsv[:, :, 1] - hsv[:, :, 1].min()) / (hsv[:, :, 1].max() - hsv[:, :, 1].min()) * 0.8 + 0.2
    rgb = skimage.color.hsv2rgb(hsv)
    rgb = rgb * 255
    rgb = rgb.astype(np.uint8)
    com = cv2.polylines(rgb.copy(), [polygon.astype(np.int32)], True, [0, 255, 0], 15, lineType=8)
    filename = subpath + structure + '_' + str(section) + '.tif'
    cv2.imwrite(os.environ['ROOT_DIR']+filename, com)
    print('Image finished in %5.1f seconds' % (time() - t5))
    
    t6=time()
    setup_upload_from_s3(filename, recursive=False)
    time_upload=time_upload+time()-t6
    count += 1
    print(section, structure, count, '/', len(polygons), time()-t0)
# if NotUpload:
#     pickle.dump(grid_features, open(os.environ['ROOT_DIR'] + grid_fn, 'wb'))
#     setup_upload_from_s3(grid_fn, recursive=False)
#os.remove(os.environ['ROOT_DIR']+img_fn)
print(str(section) + ' finished in %5.1f seconds' % (time() - tb))

Train finished in   0.8 seconds
location finished in   0.0 seconds
Features finished in  79.8 seconds
Allocation finished in  80.3 seconds
Image finished in   6.5 seconds
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/SNR/SNR_130.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/SNR/SNR_130.tif
130 SNR 1 / 7 110.2704930305481
Train finished in   0.9 seconds
location finished in   0.0 seconds
Features finished in  67.4 seconds
Allocation finished in  67.9 seconds
Image finished in   4.7 seconds
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/DC/DC_130.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/DC/DC_130.tif
130 DC 2 / 7 95.47600388526917
Train finished in   0.7 seconds
location finished in   0.0 seconds
Features finished in  70.9 seconds
Allocation finished in  71.4 seconds
Image finished in   4.6 seconds
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/Sp5I/Sp5I_130.tif s3://mousebrainatlas-

In [12]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['contour', 'f', 'hsv', 'stack', 'time', 'negative']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [14]:
time_DM

0

In [21]:
num_cells/time_features

624.2383833522377

In [10]:
print('Features:   %5.1f s'% (time_features)) 
print('Number of Patches:', num_patch)
print('Average time/Patch: %5.1f ms'% (time_features/num_patch*1000))
print('Compute DMs:        %5.1f s, %5.3f'% (time_DMs, time_DMs/time_features))
print('Transform Process:  %5.1f s, %5.3f'% (time_transforms, time_transforms/time_features))

Features:   381.8 s
Number of Patches: 1224
Average time/Patch: 311.9 ms
Compute DMs:        355.2 s, 0.930
Transform Process:  353.3 s, 0.925


In [22]:
224*0.46

103.04

In [14]:
%%time
grid_features = {}
NotUpload = True
count = 0
tb = time()
time_train = 0
time_upload = 0
time_features = 0
time_DMs = 0
time_transforms = 0
num_patch = 0
for contour_id, contour in polygons:
    t0=time()
    structure = contour_id
    if structure not in all_patch_locations[section].keys():
        continue
    polygon = contour.copy()

    if structure == '7n':
        structure = '7nn'

    subpath = savepath + structure + '/'
    if not os.path.exists(os.environ['ROOT_DIR']+subpath):
        os.mkdir(os.environ['ROOT_DIR']+subpath)
    
    t1=time()
    fp = []
    fp.append(cell_dir + structure + '/MD589_' + structure + '_positive.pkl')
    fp.append(cell_dir + structure + '/MD589_' + structure + '_negative.pkl')
    features = []
    labels = []
    for state in range(2):
        clouds = pickle.load(open(fp[state], 'rb'))
        features.extend(np.array(clouds))
        labels.extend([1 - state] * len(clouds))
    features = np.array(features)
    labels = np.array(labels)
    X_train = features
    y_train = labels
    dtrain = xgb.DMatrix(X_train, label=y_train)
    bst = xgb.train(param, dtrain, num_round, verbose_eval=False)
    time_train = time_train+time()-t1
    print('Train finished in %5.1f seconds' % (time() - t1))
    

    if structure == '7nn':
        structure = '7n'

    negative = structure + '_surround_500um_noclass'

    [left, right, up, down] = [int(max(min(all_patch_locations[section][negative][:, 0]) - half_size, 0)),
                               int(min(np.ceil(max(all_patch_locations[section][negative][:, 0]) + half_size),
                                       n - 1)),
                               int(max(min(all_patch_locations[section][negative][:, 1]) - half_size, 0)),
                               int(min(np.ceil(max(all_patch_locations[section][negative][:, 1]) + half_size),
                                       m - 1))]
    t2=time()
    xs, ys = np.meshgrid(np.arange(left + half_size, right - half_size + 1, half_size * 2),
                         np.arange(up + half_size, down - half_size + 1, half_size * 2), indexing='xy')
    locations = np.c_[xs.flat, ys.flat]
    inside = all_patch_locations[section][structure]
    all_rows = locations.view([('', locations.dtype)] * locations.shape[1])
    inside_rows = inside.view([('', inside.dtype)] * inside.shape[1])
    outside = np.setdiff1d(all_rows, inside_rows).view(locations.dtype).reshape(-1, locations.shape[1])
    windows = []
    windows.append(inside)
    windows.append(outside)
    polygon[:, 0] = polygon[:, 0] - left
    polygon[:, 1] = polygon[:, 1] - up
    print('location finished in %5.1f seconds' % (time() - t2))

    hsv = np.zeros([down - up + 1, right - left + 1, 3])
    hsv[:, :, 2] = 1
    single_time = 0
    t3=time()
    for state in range(2):
        for index in range(len(windows[state])):
            try:
                t4=time()
                x = int(float(windows[state][index][0]))
                y = int(float(windows[state][index][1]))
                patch = img[y - half_size:y + half_size, x - half_size:x + half_size].copy()
                grid_index = str(section)+'_'+str(x)+'_'+str(y)
                if grid_index in grid_features.keys():
                    extracted = grid_features[grid_index]
                else:
                    extracted, time_DM, time_transform = features_extractor(patch, params)
                    grid_features[grid_index] = extracted
                    time_DMs += time_DM
                    time_transforms += time_transform
                    num_patch += 1
                single_time=single_time+time()-t4
                
                xtest = xgb.DMatrix(extracted)
                score = bst.predict(xtest, output_margin=True, ntree_limit=bst.best_ntree_limit)
                value_img = patch / 255
                hsv[y - half_size - up:y + half_size - up, x - half_size - left:x + half_size - left, 2] = value_img
                satua_img = np.zeros_like(value_img) + score
                origin = hsv[y - half_size - up:y + half_size - up, x - half_size - left:x + half_size - left, 1]
                comp = np.absolute(origin) - np.absolute(satua_img)
                hsv[y - half_size - up:y + half_size - up, x - half_size - left:x + half_size - left, \
                1] = origin * (comp > 0) + satua_img * (comp < 0)
            except:
                continue
    time_features=time_features+single_time
    print('Features finished in %5.1f seconds' % (single_time))
    print('Allocation finished in %5.1f seconds' % (time() - t3))
    t5=time()
    hsv[:, :, 0] = (hsv[:, :, 1] < 0) * 0.66 + (hsv[:, :, 1] > 0) * 1.0
    hsv[:, :, 1] = np.absolute(hsv[:, :, 1])
    hsv[:, :, 1] = (hsv[:, :, 1] - hsv[:, :, 1].min()) / (hsv[:, :, 1].max() - hsv[:, :, 1].min()) * 0.8 + 0.2
    rgb = skimage.color.hsv2rgb(hsv)
    rgb = rgb * 255
    rgb = rgb.astype(np.uint8)
    com = cv2.polylines(rgb.copy(), [polygon.astype(np.int32)], True, [0, 255, 0], 15, lineType=8)
    filename = subpath + structure + '_' + str(section) + '.tif'
    cv2.imwrite(os.environ['ROOT_DIR']+filename, com)
    print('Image finished in %5.1f seconds' % (time() - t5))
    
    t6=time()
    setup_upload_from_s3(filename, recursive=False)
    time_upload=time_upload+time()-t6
    count += 1
    print(section, structure, count, '/', len(polygons), time()-t0)
# if NotUpload:
#     pickle.dump(grid_features, open(os.environ['ROOT_DIR'] + grid_fn, 'wb'))
#     setup_upload_from_s3(grid_fn, recursive=False)
os.remove(os.environ['ROOT_DIR']+img_fn)
print(str(section) + ' finished in %5.1f seconds' % (time() - tb))

Train finished in   0.8 seconds
location finished in   0.0 seconds
Features finished in  93.4 seconds
Allocation finished in  93.9 seconds
Image finished in   6.7 seconds
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/SNR/SNR_130.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/SNR/SNR_130.tif
130 SNR 1 / 7 127.13819003105164
Train finished in   0.8 seconds
location finished in   0.0 seconds
Features finished in  78.2 seconds
Allocation finished in  78.7 seconds
Image finished in   5.1 seconds
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/DC/DC_130.tif s3://mousebrainatlas-data/CSHL_hsv/MD594/DC/DC_130.tif
130 DC 2 / 7 107.2343738079071
Train finished in   0.7 seconds
location finished in   0.0 seconds
Features finished in  82.7 seconds
Allocation finished in  83.2 seconds
Image finished in   5.4 seconds
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD594/Sp5I/Sp5I_130.tif s3://mousebrainatlas

In [16]:
total = 633.2
print('Train:  ', time_train, time_train/total)
print('Upload:', time_upload, time_upload/total)
print('Features:', time_features, time_features/total)

Train: 4.019002199172974 0.006347129183785492
Upload: 152.4430992603302 0.24075031468782404
Features: 439.1544015407562 0.6935476966847065


In [24]:
print('Features:   %5.1f s'% (time_features)) 
print('Number of Patches:', num_patch)
print('Average time/Patch: %5.1f ms'% (time_features/num_patch*1000))
print('Compute DMs:        %5.1f s, %5.3f'% (time_DMs, time_DMs/time_features))
print('Transform Process:  %5.1f s, %5.3f'% (time_transforms, time_transforms/time_features))

Features:   439.2 s
Number of Patches: 1224
Average time/Patch: 358.8 ms
Compute DMs:        406.8 s, 0.926
Transform Process:  346.0 s, 0.788


In [40]:
%%time
x = int(float(windows[1][5][0]))
y = int(float(windows[1][5][1]))
patch = img[y - half_size:y + half_size, x - half_size:x + half_size].copy()
extractor=patch_extractor(patch,params)
tile=patch #cv2.imread(patch,0)
if params['preprocessing']['polarity']==-1:
    tile = 255-tile
min_std=params['preprocessing']['min_std']
_std = np.std(tile.flatten())

extracted = []
if _std < min_std:
    extracted.append([0] * 201)
else:
    Stats = extractor.segment_cells(tile)
    cells = extractor.extract_blobs(Stats,tile)
    cells = pd.DataFrame(cells)
    cells = cells[cells['padded_patch'].notnull()]
    cells = cells.drop(['padded_patch','left','top'],1)
    cells = np.asarray(cells)
    for k in range(len(cells)):
        cells[k][0] = cells[k][0][:10]
    origin = np.concatenate((np.array(list(cells[:,0])),cells[:,1:]),axis=1)
    for k in range(origin.shape[1]):
        x, y = CDF(origin[:,k])
        ten = [x[np.argmin(np.absolute(y-0.1*(j+1)))] for j in range(10)]
        extracted.extend(ten)
    extracted.extend([cells.shape[0]/100])

CPU times: user 141 ms, sys: 52.7 ms, total: 194 ms
Wall time: 410 ms


In [41]:
extractor.timestamps[-1][1]-extractor.timestamps[0][1]

0.3839700222015381

In [43]:
time = 0
for i in range(1,len(extractor.timestamps)-1,2):
    time=time+extractor.timestamps[i+1][1]-extractor.timestamps[i][1]
time

0.3173530101776123